In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!apt-get update
!apt install chromium-chromedriver
!pip install selenium==4.26.1

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,172 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,734 kB]
Get:13 http://archive.ubuntu.com/ubuntu jamm

In [7]:
# (최초 1회)
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver '/content/drive/MyDrive/Colab Notebooks' #
!pip install chromedriver-autoinstaller

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
chromium-chromedriver is already the newest version (1:85.0.4183.83-0ubuntu2.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 48 not upgraded.
cp: cannot create regular file '/content/drive/MyDrive/Colab Notebooks': No such file or directory


In [8]:
!python --version

import selenium
print("selenium",selenium.__version__)

Python 3.10.12
selenium 4.26.1


In [57]:
# Import modules
import chromedriver_autoinstaller
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from google.colab import drive
import os
import json

In [41]:
# Install the correct version of chromedriver
chromedriver_autoinstaller.install()

In [16]:
# Set up the Chrome driver
options = Options()
options.add_argument('--headless')  # Run in headless mode
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

In [89]:
driver = webdriver.Chrome(options=options)

In [90]:
# Open the YouTube channel's videos page
driver.get("https://www.youtube.com/@DonaldJTrumpforPresident/videos")

In [91]:
# Scroll and collect hrefs
SCROLL_PAUSE_TIME = 2
NUM_VIDEOS_TO_COLLECT = 150

In [92]:
hrefs = []
last_height = driver.execute_script("return document.documentElement.scrollHeight")

In [105]:
video_data = []
while len(video_data) < NUM_VIDEOS_TO_COLLECT:
    # Scroll down to the bottom
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")

    # Wait for new content to load
    time.sleep(SCROLL_PAUSE_TIME)

    # Find all video elements
    elements = driver.find_elements(By.CSS_SELECTOR, 'ytd-rich-grid-media')
    for element in elements:
        # Extract title
        title_element = element.find_element(By.CSS_SELECTOR, 'yt-formatted-string#video-title')
        title = title_element.text

        # Extract URL
        link_element = element.find_element(By.CSS_SELECTOR, 'a.yt-simple-endpoint.inline-block.style-scope.ytd-thumbnail')
        href = link_element.get_attribute('href')

        # Add to video data if unique
        if href and not any(video['url'] == href for video in video_data):
            video_data.append({
                'title': title,
                'url': href
            })

    # Check if we've reached the bottom of the page
    new_height = driver.execute_script("return document.documentElement.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

In [94]:
video_data

[{'title': 'It’s Time.', 'url': 'https://www.youtube.com/watch?v=YY5BPigFKgs'},
 {'title': 'Together.', 'url': 'https://www.youtube.com/watch?v=knkdSsAlDhc'},
 {'title': 'Tomorrow.', 'url': 'https://www.youtube.com/watch?v=A3Wy6x3DohI'},
 {'title': 'A Special Message from President Donald J. Trump',
  'url': 'https://www.youtube.com/watch?v=XSZS8idtKmI'},
 {'title': 'We Fight', 'url': 'https://www.youtube.com/watch?v=uP08OUoVj0o'},
 {'title': 'Trump Force 47 - Weekly Recap',
  'url': 'https://www.youtube.com/watch?v=d608OHI_muM'},
 {'title': 'MAGA Women Respond to Kamala Surrogate Mark Cuban',
  'url': 'https://www.youtube.com/watch?v=BGyVZePe7Ow'},
 {'title': "I'm Not With Her",
  'url': 'https://www.youtube.com/watch?v=z1Awk3PSIJo'},
 {'title': 'President Trump Tells the Hilarious Story of How the Garbage Truck and the Orange Vest Came To Be',
  'url': 'https://www.youtube.com/watch?v=ckXcyoOPP3A'},
 {'title': 'Kamala and Joe call us all garbage. I call you the heart and soul of Amer

In [95]:
# Prepare save path
save_path = '/content/drive/MyDrive/[인공지능기초]/sns/youtube'
os.makedirs(save_path, exist_ok=True)
file_path = os.path.join(save_path, 'trump_youtube_url_title.json')

In [96]:
# Save the video data to a JSON file
with open(file_path, 'w') as f:
    json.dump(video_data, f, ensure_ascii=False, indent=4)

In [22]:
# Prepare the list of video URLs
urls = []
for href in hrefs:
    if href.startswith('/'):
        full_url = 'https://www.youtube.com' + href
    else:
        full_url = href
    urls.append(full_url)

In [25]:
urls = urls[:150]

In [28]:
len(urls)

150

In [27]:
save_path = '/content/drive/MyDrive/[인공지능기초]/sns/youtube'
os.makedirs(save_path, exist_ok=True)
file_path = os.path.join(save_path, 'trump_youtube_url.json')

In [29]:
# Save the URLs to a JSON file
with open(file_path, 'w') as f:
    json.dump(urls, f, ensure_ascii=False, indent=4)

In [ ]:
# 7월 21부터 최근까지 영상개수가 150개 이하로 추정
# 정확한 날짜로 필터링 하기 위해서는 해당 url을 모두 접속하여 업로드 날짜를 스크래핑하고 필터링하도록 한다.

In [33]:
with open('/content/drive/MyDrive/[인공지능기초]/sns/youtube/trump_youtube_url.json', 'r') as f:
    urls = json.load(f)

In [61]:
dates = []

In [63]:
for url in urls:
    driver.get(url)

    # 페이지 로딩 대기
    wait = WebDriverWait(driver, 10)
    time.sleep(2)  # 추가 대기 시간

    # "설명" 섹션 클릭
    try:
        description_div = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'div#description-inner')))
        driver.execute_script("arguments[0].click();", description_div)
        time.sleep(1)  # 클릭 후 대기
    except Exception as e:
        print(f'"설명" 섹션을 클릭할 수 없습니다: {url}')
        print(f'에러: {e}')
        continue  # 다음 URL로 진행

    # 날짜 스크래핑
    try:
        # 날짜 정보가 로드될 때까지 대기
        date_element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'yt-formatted-string#info')))
        spans = date_element.find_elements(By.TAG_NAME, 'span')

        # 날짜 추출 (마지막 span이 날짜를 포함한다고 가정)
        if spans:
            date_text = spans[-1].text.strip()
            dates.append({'url': url, 'date': date_text})
            print(f'날짜를 찾았습니다: {date_text}')
        else:
            print(f'날짜를 찾을 수 없습니다: {url}')
    except Exception as e:
        print(f'날짜를 찾을 수 없습니다: {url}')
        print(f'에러: {e}')

날짜를 찾았습니다: Nov 16, 2024
날짜를 찾았습니다: Nov 4, 2024
날짜를 찾았습니다: Nov 4, 2024
날짜를 찾았습니다: Premiered Nov 3, 2024
날짜를 찾았습니다: Nov 2, 2024
날짜를 찾았습니다: Nov 1, 2024
날짜를 찾았습니다: Nov 1, 2024
날짜를 찾았습니다: Nov 1, 2024
날짜를 찾았습니다: Oct 31, 2024
날짜를 찾았습니다: Oct 31, 2024
날짜를 찾았습니다: Oct 30, 2024
날짜를 찾았습니다: Oct 30, 2024
날짜를 찾았습니다: Oct 30, 2024
날짜를 찾았습니다: Oct 30, 2024
날짜를 찾았습니다: Oct 30, 2024
날짜를 찾았습니다: Oct 28, 2024
날짜를 찾았습니다: Oct 27, 2024
날짜를 찾았습니다: Oct 26, 2024
날짜를 찾았습니다: Oct 25, 2024
날짜를 찾았습니다: Oct 25, 2024
날짜를 찾았습니다: Oct 24, 2024
날짜를 찾았습니다: Oct 24, 2024
날짜를 찾았습니다: Oct 23, 2024
날짜를 찾았습니다: Oct 23, 2024
날짜를 찾았습니다: Oct 23, 2024
날짜를 찾았습니다: Oct 22, 2024
날짜를 찾았습니다: Oct 20, 2024
날짜를 찾았습니다: Oct 20, 2024
날짜를 찾았습니다: Oct 19, 2024
날짜를 찾았습니다: Oct 18, 2024
날짜를 찾았습니다: Oct 18, 2024
날짜를 찾았습니다: Oct 18, 2024
날짜를 찾았습니다: Oct 18, 2024
날짜를 찾았습니다: Premiered Oct 18, 2024
날짜를 찾았습니다: Oct 17, 2024
날짜를 찾았습니다: Oct 16, 2024
날짜를 찾았습니다: Oct 16, 2024
날짜를 찾았습니다: Oct 16, 2024
날짜를 찾았습니다: Oct 16, 2024
날짜를 찾았습니다: Oct 15, 2024
날짜를 찾았습니다: Oct 14, 2024
날짜를

In [64]:
# 결과를 JSON 파일로 저장
output_path = '/content/drive/MyDrive/[인공지능기초]/sns/youtube/dates.json'
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(dates, f, ensure_ascii=False, indent=4)

In [71]:
with open('/content/drive/MyDrive/[인공지능기초]/sns/youtube/trump_youtube_url_date.json', 'r') as f:
    urls = json.load(f)

In [72]:
from datetime import datetime

In [73]:
# Define the date range
start_date = datetime.strptime("Jul 21, 2024", "%b %d, %Y")
end_date = datetime.strptime("Nov 4, 2024", "%b %d, %Y")

In [74]:
# Function to parse the date and check if it falls within the desired range
def is_within_range(date_str):
    # Handle "Premiered" date format
    if "Premiered" in date_str:
        date_str = date_str.replace("Premiered", "").strip()

    # Try parsing the date
    try:
        # Try multiple formats for flexibility
        date_obj = datetime.strptime(date_str, "%b %d, %Y")
    except ValueError:
        try:
            date_obj = datetime.strptime(date_str, "%b %d, %Y")
        except ValueError:
            return False

    # Check if the date is within the desired range
    return start_date <= date_obj <= end_date

In [75]:
# Filter the data
filtered_urls = [url for url in urls if is_within_range(url['date'])]

In [77]:
len(filtered_urls)

113

In [78]:
# Save the filtered data back to the file
with open('/content/drive/MyDrive/[인공지능기초]/sns/youtube/trump_youtube_url_date_filtered.json', 'w') as f:
    json.dump(filtered_urls, f, indent=4)

In [79]:
driver.quit()

In [97]:
# title을 깜빡 잊어서 title 추가 후 수정

In [99]:
# Load trump_youtube_url_date_filtered.json
with open("/content/drive/MyDrive/[인공지능기초]/sns/youtube/trump_youtube_url_date_filtered.json", "r") as date_file:
    date_data = json.load(date_file)

# Load trump_youtube_url_title.json
with open("/content/drive/MyDrive/[인공지능기초]/sns/youtube/trump_youtube_url_title.json", "r") as title_file:
    title_data = json.load(title_file)

# Create a dictionary from title_data for quick lookup by URL
title_dict = {item["url"]: item["title"] for item in title_data}

# Add titles to date_data if the URL matches
for item in date_data:
    if item["url"] in title_dict:
        item["title"] = title_dict[item["url"]]

# Save the updated date_data to a new file
with open("trump_youtube_url_date_filtered_with_titles.json", "w") as output_file:
    json.dump(date_data, output_file, ensure_ascii=False, indent=4)

In [100]:
with open("trump_youtube_url_date_filtered_with_titles.json", "r") as date_file:
    su_data = json.load(date_file)

In [103]:
len(su_data)

113

In [104]:
su_data

[{'url': 'https://www.youtube.com/watch?v=knkdSsAlDhc',
  'date': 'Nov 4, 2024',
  'title': 'Together.'},
 {'url': 'https://www.youtube.com/watch?v=A3Wy6x3DohI',
  'date': 'Nov 4, 2024',
  'title': 'Tomorrow.'},
 {'url': 'https://www.youtube.com/watch?v=XSZS8idtKmI',
  'date': 'Premiered Nov 3, 2024',
  'title': 'A Special Message from President Donald J. Trump'},
 {'url': 'https://www.youtube.com/watch?v=uP08OUoVj0o',
  'date': 'Nov 2, 2024',
  'title': 'We Fight'},
 {'url': 'https://www.youtube.com/watch?v=d608OHI_muM',
  'date': 'Nov 1, 2024',
  'title': 'Trump Force 47 - Weekly Recap'},
 {'url': 'https://www.youtube.com/watch?v=BGyVZePe7Ow',
  'date': 'Nov 1, 2024',
  'title': 'MAGA Women Respond to Kamala Surrogate Mark Cuban'},
 {'url': 'https://www.youtube.com/watch?v=z1Awk3PSIJo',
  'date': 'Nov 1, 2024',
  'title': "I'm Not With Her"},
 {'url': 'https://www.youtube.com/watch?v=ckXcyoOPP3A',
  'date': 'Oct 31, 2024',
  'title': 'President Trump Tells the Hilarious Story of How 